In [1]:
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
from skimage.measure import label, regionprops
import skimage
from skimage.io import imread,imsave
from skimage.filters import threshold_otsu
import numpy as np
from skimage import morphology
import warnings
warnings.filterwarnings('ignore')
import math
from Models import DualEncoding_U_Net,load_model

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
path_h_gray='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/H_gray'
path_h_e='/home/vahadaneabhi01/datalab/training-assets/R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/Images'

model=DualEncoding_U_Net(img1_ch=3,img2_ch=1,output_ch=2,dropout=0.3,include_ffm=False)     # Define the model
filename='model_CPM_17_2020_02_06/model_optim.pth'
load_model(filename,model,optimizer=None,scheduler=None) 


FileNotFoundError: [Errno 2] No such file or directory: 'model_CPM_17_2020_02_06/model_optim.pth'

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
a=imread('/home/vahadaneabhi01/datalab/training-assets/\
R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/Images/gbm_image03.png')[250-128:250+128,250-128:250+128,:3]/255

b=imread('/home/vahadaneabhi01/datalab/training-assets/\
R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/NucleiMaps/gbm_image03.png')[250-128:250+128,250-128:250+128]/255

c=1-imread('/home/vahadaneabhi01/datalab/training-assets/\
R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/H_gray/gbm_image03.png')[250-128:250+128,250-128:250+128]/255
plt.imshow(c)
a=torch.from_numpy(np.expand_dims(a.transpose((2, 0, 1)),axis=0)).type(torch.FloatTensor).to(device)
b=torch.from_numpy(np.expand_dims(np.expand_dims(b,axis=2).transpose((2, 0, 1)),axis=0))\
                   .type(torch.FloatTensor).to(device)
c=torch.from_numpy(np.expand_dims(np.expand_dims(c,axis=2).transpose((2, 0, 1)),axis=0))\
                   .type(torch.FloatTensor).to(device)
print(a.size())
output,list_attn=model(a,c)

In [ ]:
print(list_attn[2][0][0].detach().cpu().numpy().shape)
plt.imshow(list_attn[2][0][0].detach().cpu().numpy()*255,cmap='YlGnBu')

In [ ]:
print(list_attn[2][0][0].detach().cpu().numpy())

In [ ]:
k=imread('/home/vahadaneabhi01/datalab/training-assets/\
R_medical/atheeth/nuclei_seg/other_data/CPM_17/Test/Images/gbm_image05.png')
print(k.shape)
plt.imshow(k)

In [ ]:
a=imread('nuclei_TCGA-AY-A8YK-01A-01-TS1 (1).png')
b=imread('bound_TCGA-AY-A8YK-01A-01-TS1.png')
plt.imshow(b)

In [5]:
def whole_dice_metric(y_pred,y_true):
    smooth = 10e-16
    # single image so just roll it out into a 1D array
    
    m1 =np.reshape(y_pred,(-1))/255
    m2 =np.reshape(y_true,(-1))/255
    
    
    intersection = (m1 * m2)

    score = 2. * (np.sum(intersection) + smooth) / (np.sum(m1) +(np.sum(m2) + smooth))
        
    return score

def multiple_erosion(img,iter=5):
    for j in (range(5)):
        img=morphology.binary_erosion(img, selem=morphology.selem.disk(1))
    return img

def multiple_dialte(img,iter=5):
    for j in (range(5)):
        img=morphology.binary_dilation(img, selem=morphology.selem.disk(1))
    return img

def coord2array(coord):
    x=[]
    y=[]
    for i in coord:
        x.append(i[0])
        y.append(i[1])
    return (x,y)


In [6]:
def img_recon(y_pred,bound_img,boundary_length=150,iter_limt=2,print_prompt=True):
    smooth=1
    
    # args:
    #y_pred : thresholded and post processed nuclei seeds.
    #bound_img : raw_boundary predicitons.
    #n_array_gt : GT N-array mask
    #boundary_length : hyperparameter for growing nuclei seeds
    #iter_limt : hyperparameter for growing nuclei seeds
    #print_prompt : wheter or not to show progress bar
    thresh_bound=0.2*255
    bound_img=bound_img>thresh_bound
    bound_img=multiple_dialte(bound_img,iter=3)
    bound_img=multiple_erosion(bound_img,iter=3)

    labeled_img=label(y_pred)
    regions=regionprops(labeled_img)

    final_image=np.zeros_like(y_pred,dtype=np.uint16)
    if print_prompt:
        loop=tqdm(regions)
    else:
        loop=regions
    
    for i,region in enumerate(loop):
        coordinates=coord2array(list(region.coords))
        x_temp=np.zeros_like(y_pred)
        x_temp[coordinates]=1
        boundary_score=0
        iteration=0
        while boundary_score<boundary_length and iteration<iter_limt:
            iteration+=1
            x_temp=morphology.binary_dilation(x_temp, selem=morphology.selem.disk(2)).astype(np.float)
            boundary_score=len(np.where(x_temp*bound_img==1)[0])
        
        
        final_image[np.where(x_temp==1)]=i
    
 
        
    return final_image

In [ ]:

img_list=[x for x in os.listdir('prediction_kumar') if 'nuc' in x and 'proc' not in x]
avg=0
for img_name in tqdm(img_list):
    nuclei=imread('prediction_kumar/'+img_name)
    thresh=threshold_otsu(nuclei)
    nuclei=nuclei>thresh
    nuclei=multiple_erosion(nuclei,iter=3)
    
    boundary=imread('prediction_kumar/'+'bound_'+'_'.join(img_name.split('_')[1:]))
    
    gt=imread('processed_data/nucleus_maps/'+img_name.split('.')[0].split('_')[1]+'_nucleus_mask.png')
    img_pred=img_recon(nuclei,boundary,boundary_length=150,iter_limt=2,print_prompt=False)
    
    imsave('prediction_kumar/'+img_name.split('.')[0]+'_proc.png',img_pred.astype(np.float32))
    
    img_pred=img_pred>0
    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0].split('_')[1],' : ',dice,thresh)
print(avg/len(img_list))

TCGA-E2-A14V-01Z-00-DX1  :  0.8225290612209419<br/>
TCGA-21-5786-01Z-00-DX1  :  0.6991805205647318<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7702200475923103<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.7629844577597736<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7624224458621036<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.7970872142252329<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8176449329279959<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.7966429519738157<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8649618223341795<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8129848869951398<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7642849375366202<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.7747906286441217<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7865019409500186<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.6759419369045772<br/>
0.77915555610654<br/>
# Model 30th Jan 2020 model_optim.pth
TCGA-21-5784-01Z-00-DX1  :  0.8116361668301593 108<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8782868692362495 117<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.815197744362616 115<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.7743249219687112 114<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.7873815363001121 108<br/>
TCGA-G9-6348-01Z-00-DX1  :  0.8273699189706346 112<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8798372531910145 118<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.883796541868124 115<br/>
TCGA-21-5786-01Z-00-DX1  :  0.7315855593758265 111<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.8599896627535161 113<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7949021136999774 113<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.802531826634244 110<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.788953461927371 105<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.828047288316124 109<br/>
0.8188457761024771<br/>

# Model 31st 2 channel pred with only otsu
TCGA-G9-6348-01Z-00-DX1  :  0.7819525407623568 108<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.8811115372673644 122<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.8138068067885593 111<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8401957024486049 118<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7201258488319375 124<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.8268603691132442 107<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.7434148978191721 94<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7782766771403891 108<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.7851957232741299 106<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8711696861502439 120<br/>
TCGA-21-5786-01Z-00-DX1  :  0.7051864689168885 107<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.8391341602292577 113<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.6983577090485372 110<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.6546214106691893 126<br/>
0.7813863956042768<br/>
# Model 31st 2 channel pred with boundary constratined growing of nuclei seeds
TCGA-G9-6348-01Z-00-DX1  :  0.7193487672267217 108<br/>
TCGA-KB-A93J-01A-01-TS1  :  0.7978143579195152 122<br/>
TCGA-CH-5767-01Z-00-DX1  :  0.7869126701887118 111<br/>
TCGA-DK-A2I6-01A-01-TS1  :  0.8429467108874303 118<br/>
TCGA-NH-A8F7-01A-01-TS1  :  0.7064353205931725 124<br/>
TCGA-B0-5710-01Z-00-DX1  :  0.7426428449341953 107<br/>
TCGA-HE-7128-01Z-00-DX1  :  0.6464007628952234 94<br/>
TCGA-21-5784-01Z-00-DX1  :  0.7619342935549388 108<br/>
TCGA-E2-A1B5-01Z-00-DX1  :  0.751238314651544 106<br/>
TCGA-RD-A8N9-01A-01-TS1  :  0.8106484504188964 120<br/>
TCGA-21-5786-01Z-00-DX1  :  0.5648986336791768 107<br/>
TCGA-E2-A14V-01Z-00-DX1  :  0.815677097344578 113<br/>
TCGA-G2-A2EK-01A-02-TSB  :  0.760678717070453 110<br/>
TCGA-AY-A8YK-01A-01-TS1  :  0.6709322769456747 126<br/>
0.7413220870221593<br/>

In [ ]:

img_list=[x for x in os.listdir('prediction_CoNSep') if 'test' in x and 'proc' not in x]
avg=0
for img_name in img_list:
    img_pred=imread('prediction_CoNSep/'+img_name)
    gt=np.load('other_data/CoNSeP/Test/Labels/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.npy')[:,:,0]
    gt=(gt>0).astype(np.uint8)*255
    
    thresh=threshold_otsu(img_pred)
    img_pred=img_pred>thresh
    
    imsave('prediction_CoNSep/'+img_name.split('.')[0]+'_proc.png',img_pred.astype(np.float32))

    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0],' : ',dice,thresh)
print(avg/len(img_list))

In [7]:
img_list=[x for x in os.listdir('prediction_CPM_17') if 'nuclei' in x and 'proc' not in x]
avg=0
for img_name in img_list:
    
    img_pred=imread('prediction_CPM_17/'+img_name)
    gt=imread('other_data/CPM_17/Test/NucleiMaps/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.png')
    
    
    thresh=threshold_otsu(img_pred)
    img_pred=img_pred>thresh
    
#     imsave('prediction_CPM_17/'+img_name.split('.')[0]+'_proc.png',img_pred.astype(np.float32))

    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0],' : ',dice,thresh)
print(avg/len(img_list))


nuclei_gbm_image08  :  0.9004523281596453 120
nuclei_lgg_image03  :  0.879105031109646 115
nuclei_lgg_image08  :  0.8615869570834302 117
nuclei_gbm_image04  :  0.8934956209744733 117
nuclei_lung_image03  :  0.8536544432323572 119
nuclei_hnsc_image07  :  0.8329886978443758 116
nuclei_lgg_image05  :  0.8849932894802247 111
nuclei_lgg_image06  :  0.8554616852067487 112
nuclei_lung_image01  :  0.6672766502012327 108
nuclei_hnsc_image08  :  0.8253571536898299 114
nuclei_gbm_image03  :  0.8489450763181261 113
nuclei_lung_image05  :  0.9113640224222864 117
nuclei_hnsc_image06  :  0.7976550969889047 107
nuclei_lung_image06  :  0.9075399796421391 115
nuclei_gbm_image06  :  0.8662990874726122 112
nuclei_lung_image02  :  0.7927961982935522 118
nuclei_lgg_image01  :  0.8506631247150249 115
nuclei_hnsc_image03  :  0.866154184658544 119
nuclei_gbm_image07  :  0.8810295957713031 118
nuclei_lgg_image02  :  0.8675806247323566 117
nuclei_lung_image07  :  0.9173411265667449 117
nuclei_hnsc_image01  :  0.

In [ ]:
img_list=[x for x in os.listdir('prediction_CPM_17') if 'nuclei' in x and 'proc' not in x]
avg=0
for img_name in img_list:
    
    nuclei=imread('prediction_CPM_17/'+img_name)
    thresh=threshold_otsu(nuclei)
    nuclei=nuclei>thresh
    nuclei=multiple_erosion(nuclei,iter=1)
    
    boundary=imread('prediction_CPM_17/'+'bound_'+'_'.join(img_name.split('_')[1:]))
    
    thresh_bound=threshold_otsu(boundary)
    boundary=boundary>thresh_bound
#     boundary=multiple_dialte(boundary,iter=2)
    
    gt=imread('other_data/CPM_17/Test/NucleiMaps/'+'_'.join(img_name.split('_')[1:]).split('.')[0]+'.png')
    
#     img_pred=img_recon(nuclei,boundary,boundary_length=150,iter_limt=2,print_prompt=False)
    img_pred=nuclei.astype(np.uint8)-boundary.astype(np.uint8)
#     img_pred=multiple_dialte(img_pred,iter=1)
    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    imsave('prediction_CPM_17/'+img_name.split('.')[0]+'_proc.png',img_pred.astype(np.float32))

    dice=whole_dice_metric(img_pred.astype(np.float32)*255,gt)
    avg+=dice
    print(img_name.split('.')[0],' : ',dice,thresh)
print(avg/len(img_list))


Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


nuclei_gbm_image08  :  1.045108307950774 120


Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


nuclei_lgg_image03  :  0.9928117276847764 115


Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


...

Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


nuclei_hnsc_image07  :  0.30367045603922943 116


Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


nuclei_lgg_image05  :  0.5741601059816842 111


Lossy conversion from float32 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


# model 4th feb, predicts only nuclei

nuclei_gbm_image01  :  0.7277193766051461 108<br/>
nuclei_lgg_image02  :  0.8795810296965031 120<br/>
nuclei_lgg_image04  :  0.8498570690545892 120<br/>
nuclei_gbm_image05  :  0.8793331994376381 115<br/>
nuclei_lung_image05  :  0.9245344129554656 119<br/>
nuclei_lung_image08  :  0.8866071888262075 118<br/>
nuclei_gbm_image04  :  0.901806647358985 119<br/>
nuclei_gbm_image02  :  0.7480424652927895 110<br/>
nuclei_hnsc_image03  :  0.8753627522274856 123<br/>
nuclei_hnsc_image04  :  0.8324088699506544 121<br/>
nuclei_gbm_image08  :  0.8804250959880733 123<br/>
nuclei_lung_image06  :  0.9121998339236388 119<br/>
nuclei_lgg_image03  :  0.8737407376571477 117<br/>
nuclei_lgg_image05  :  0.8594481376923777 111<br/>
nuclei_lgg_image06  :  0.8453096210846751 113<br/>
nuclei_gbm_image06  :  0.8566324907627181 115<br/>
nuclei_gbm_image03  :  0.8456073644186511 118<br/>
nuclei_lung_image02  :  0.7613867304243873 117<br/>
nuclei_gbm_image07  :  0.8679014858538571 119<br/>
nuclei_lung_image03  :  0.8728274257561243 117<br/>
nuclei_hnsc_image01  :  0.8321472120607472 111<br/>
nuclei_hnsc_image05  :  0.8198542347323448 118<br/>
nuclei_lung_image07  :  0.9227191260561577 120<br/>
nuclei_hnsc_image06  :  0.8079416106090396 116<br/>
nuclei_lgg_image08  :  0.8623812006255764 121<br/>
nuclei_lung_image01  :  0.7257110540330373 112<br/>
nuclei_lgg_image01  :  0.8665360156781187 117<br/>
nuclei_hnsc_image02  :  0.8480114118805828 120<br/>
nuclei_hnsc_image07  :  0.841697827604559 119<br/>
nuclei_lgg_image07  :  0.8837092227144583 115<br/>
nuclei_lung_image04  :  0.8561029083091506 119<br/>
nuclei_hnsc_image08  :  0.8350518485518164 119<br/>
0.8494561127444595<br/>